# Daily Glucose Overlay

This notebook generates daily glucose overlay charts from blood glucose readings held in the Health Tracker database and retrieved via the Health Tracker Web Service:

- The median line
- A "percentile" chart shading between upper and lower percentiles

This is done for two date ranges and the data is plotted for each on the same axes for comparison purposes.

Before attempting to run the notebook:

- Make sure the variables defined in "config.ipynb" are set correctly
- Set the reporting date range and export options in the first code cell

In [ ]:
from datetime import datetime, timedelta

# Reporting dates for the first segment of data
start_range_one = datetime(2025, 1, 1, 0, 0, 0)
end_range_one = start_range_one + timedelta(days=11)

# Reporting dates for the second segment of data
start_range_two = datetime(2025, 1, 1, 0, 0, 0)
end_range_two = start_range_two + timedelta(days=11)

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
chart_export_format = "PNG"

In [ ]:
# # Set the percentile limits
# lower_percentile = 10
# upper_percentile = 90
error = 0.1

# Set the Y-axis limits
y_min = 3
y_max = 15

In [ ]:
%run ../api.ipynb
%run ../config.ipynb
%run ../export.ipynb

In [ ]:
# Log in to the service, get the person ID and retrieve the data
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)
df_range_one = get_blood_glucose_measurements(url, token, person_id, start_range_one, end_range_one)
df_range_two = get_blood_glucose_measurements(url, token, person_id, start_range_two, end_range_two)

# Preview the data
df_range_one.head()

In [ ]:
# Rename the existing date column, that contains a date and time and round the timestamp to the
# nearest 5 minutes to align the data (failing to do this means the pivot won"t work well when
# plotted)
df_range_one.rename(columns={ "date": "timestamp" }, inplace=True)
df_range_one["timestamp"] = df_range_one["timestamp"].dt.round("5min")

# Create columns for date and time-of-day
df_range_one["date"] = df_range_one["timestamp"].dt.date
df_range_one["time"] = df_range_one["timestamp"].dt.strftime("%H:%M")

# Pivot the data where rows are the time of day and columns are the date
pivot_df_range_one = df_range_one.pivot(index="time", columns="date", values="level")

# Preview the data
pivot_df_range_one.head()

In [ ]:
# Rename the existing date column, that contains a date and time and round the timestamp to the
# nearest 5 minutes to align the data (failing to do this means the pivot won"t work well when
# plotted)
df_range_two.rename(columns={ "date": "timestamp" }, inplace=True)
df_range_two["timestamp"] = df_range_two["timestamp"].dt.round("5min")

# Create columns for date and time-of-day
df_range_two["date"] = df_range_two["timestamp"].dt.date
df_range_two["time"] = df_range_two["timestamp"].dt.strftime("%H:%M")

# Pivot the data where rows are the time of day and columns are the date
pivot_df_range_two = df_range_two.pivot(index="time", columns="date", values="level")

# Preview the data
pivot_df_range_two.head()

In [ ]:
# Calculate the median lines
median_glucose_range_one = pivot_df_range_one.median(axis=1)
median_glucose_range_two = pivot_df_range_two.median(axis=1)

# # Calculate percentile bands for the shaded plot for range one
upper_range_one = median_glucose_range_one * (1.0 + error)
lower_range_one = median_glucose_range_one * (1.0 - error)

# # Calculate percentile bands for the shaded plot for range one
upper_range_two = median_glucose_range_two * (1.0 + error)
lower_range_two = median_glucose_range_two * (1.0 - error)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Set x-ticks to every hour (12 x 5 minutes = 60 minutes = 1 hour)
xticks_to_show = pivot_df_range_one.index[::12]

# Create the chart
plt.figure(figsize=(12, 8))

# # Shade the error range for range one
plt.fill_between(median_glucose_range_one.index, lower_range_one, upper_range_one, color="lightblue", alpha=0.4)

# # Shade between the percentiles for range 2
plt.fill_between(median_glucose_range_two.index, lower_range_two, upper_range_two, color="orange", alpha=0.4)

# Add the median lines
plt.plot(median_glucose_range_one.index, median_glucose_range_one.values, color="black", label="Median Range 1", linewidth=1)
plt.plot(median_glucose_range_two.index, median_glucose_range_two.values, color="red", label="Median Range 2", linewidth=1)

# Add title and labels
plt.title("Daily Glucose Overlay")
plt.xlabel("Time of Day")
plt.ylabel("Glucose (mmol/L)")
plt.ylim(y_min, y_max)
plt.xticks(ticks=xticks_to_show, rotation=90)
plt.grid(True)

# Create a legend_range_one below the plot
plt.legend(
    title="Date",
    bbox_to_anchor=(0.5, -0.2),
    loc="upper center",
    ncol=6,
    fontsize="small",
    title_fontsize="small"
)

plt.tight_layout()

# Export to PNG or PDF, if required
export_chart("glucose_overlay_comparison", "Percentile", chart_export_format)

plt.show()
